## Microbial Metabolic Interactions Tutorial

**MMinte** (pronounced /‘minti/) is a set of widgets that allows you to explore the pairwise interactions (positive or negative) that occur in a microbial community. From an association network and 16S rDNA sequence data, **MMinte** identifies corresponding genomes, reconstructs metabolic models, estimates growth under specific metabolic conditions, analyzes pairwise interactions, assigns interaction types to network links, and generates the corresponding network of interactions.

This notebook shows how to run all six widgets in an integrated pipeline from start to finish. See the Just Interactions notebook for a tutorial on how to run specific steps in the pipeline when you already have metabolic models for the organisms in a microbial community. 

### Getting started

**MMinte** can reconstruct ModelSEED models for the species in the microbial community using Widget 3. The [ModelSEED service](http://modelseed.org) uses genomes available in the [Pathosystems Resource Integration Center](https://www.patricbrc.org/portal/portal/patric/Home) (PATRIC) and requires you to authenticate as a [registered PATRIC user](http://enews.patricbrc.org/faqs/workspace-faqs/registration-faqs/). If needed, complete a [new user registration](https://user.patricbrc.org/register/). Before you use Widget 3 you must get an authentication token. The [Mackinac](https://github.com/mmundy42/mackinac) `get_token()` function stores the authentication token in the `.patric_config` file in your home directory. You can use the token until it expires.

In the cell below, change `username` to your PATRIC username and enter your password when prompted. The returned user ID identifies your ModelSEED workspace.

In [1]:
from mackinac import get_token
get_token('username')

patric password: ········


u'mmundy@patricbrc.org'

Now import the mminte package and a few other packages needed for this tutorial.

In [2]:
import mminte
from os.path import expanduser, join
from os import makedirs
import pkg_resources

Each **MMinte** widget either creates output files or returns data that is used as input to the next widget. You need to select a base folder for storing the output files and separate folders for the single species and pair model files. Since there can be a large number of model files, it helps to store them in a separate folders. Change the value assigned to `analysis_folder` if you want the output files stored somewhere else.

In [3]:
analysis_folder = join(expanduser('~'), 'mminte_tutorial')
single_model_folder = join(analysis_folder, 'single_models')
pair_model_folder = join(analysis_folder, 'pair_models')

Run the cell below to create the folders. You only need to run this cell if the folders do not exist.

In [4]:
makedirs(analysis_folder)
makedirs(single_model_folder)
makedirs(pair_model_folder)

### Widget 1 - Get sequences for representative OTUs

In Widget 1, you get the 16S rDNA sequences for the representative OTUs in the microbial community using the `get_unique_otu_sequences()` function. The first input parameter is a list of tuples that has three elements: (1) first OTU number, (2) second OTU number, and (3) correlation value between -1 and 1. You need to provide this data based on your own analysis of the microbial community.

A correlation file for this tutorial is in the test data included with the mminte package. Each line of the correlation file has three columns: (1) first OTU number, (2) second OTU number, and (3) correlation value. For example:

    18	310	0.98
    18	401	1
    18	405	0.92
    37	72	0.96	
    37	74	1
    37	632	-0.2

Each column in the correlation file is separated by a tab and the first line of the line is a header that is ignored. The correlation file is used again in Widget 6 to visualize the results.

Change the value assigned to `correlation_filename` if you want to use your own correlation file.

In [5]:
correlation_filename = pkg_resources.resource_filename('mminte', 'test/data/correlation.txt')
correlations = mminte.read_correlation_file(correlation_filename)

The second input parameter is the path to a fasta file that has the 16S rDNA sequences for all of the OTUs in the microbial community. The record ID in each fasta record must be an OTU number. All of the OTUs in the correlation file must have a record in the fasta file.

Typically you get the fasta file as output from 16S rDNA sequencing of the microbial community. A fasta file for this tutorial is in the test data included with the mminte package. The data in the fasta file comes from the Human Microbiome Project. Change the value assigned to `all_otus_filename` if you want to use your own sequence file.

In [6]:
all_otus_filename = pkg_resources.resource_filename('mminte', 'test/data/all_otus.fasta')

The third input parameter is the path to a fasta file where the 16S rDNA sequences for the unique representative OTUs are stored.

In [7]:
unique_otus_filename = join(analysis_folder, 'unique_otus.fasta')

The output is the number of unique OTUs that are found in the input files.

In [8]:
mminte.get_unique_otu_sequences(correlations, all_otus_filename, unique_otus_filename)

18

### Widget 2 - Search for closest matching bacterial species

In Widget 2, you use the unique representative OTU sequences to find the closest matching bacterial species with a whole genome sequence by running blast using the `search()` function. The first input parameter is the path to a fasta file with the 16S rDNA sequences for the unique OTUs that was created in Widget 1. The second input parameter is the path to the blast output file. *Talk about how database was generated*

The two outputs are (1) a list of the unique NCBI genome IDs and (2) a data frame with the percent similarity between each representative OTUs and the closest match, identified by the NCBI genome ID.

In [9]:
genome_ids, similarity = mminte.search(unique_otus_filename, join(analysis_folder, 'blast.txt'))

Each row in the `similarity` data frame has three columns: (1) OTU ID, (2) NCBI genome ID, and (3) percent similarity of the organism represented by the OTU number to the organism with the NCBI genome ID. For example:

In [10]:
similarity[0:10]

,OTU_ID,GENOME_ID,SIMILARITY
0,18,657315.3,100.00
1,37,484018.6,98.33
2,72,1235835.3,25.00
3,74,742823.3,100.00
4,109,1414720.3,25.22
5,112,1156417.3,90.64
6,150,180332.3,0.00
7,221,717962.3,99.16
8,243,1297617.3,91.85
9,253,1367212.3,97.24


For reference, save the genome IDs and similarity data frame to files.

In [11]:
with open(join(analysis_folder, 'genome_ids.txt'), 'w') as handle:
    handle.write('\n'.join(genome_ids)+'\n')
mminte.write_similarity_file(similarity, join(analysis_folder, 'similarity.csv'))

### Widget 3 - Reconstruct and download metabolic models from ModelSEED

In Widget 3, you reconstruct and gap fill metabolic models for the unique species in the microbial community with the [ModelSEED service](http://modelseed.org) using the `create_species_models()` function. The first input parameter is the list of genome IDs that was created in Widget 2. The second input parameter is the path to a folder for storing the single species model files (created in "Getting Started").

The output is a list of paths to the downloaded single species model files. The `create_species_models()` function can run for a long time if all of the models need to be reconstructed and gap filled. By default, `create_species_models()` uses a previously downloaded model or a previously reconstructed model available in your ModelSEED workspace. You can force the models to be reconstructed using the `replace=True` parameter.

The output is a list of paths to the downloaded single species model files.

In [12]:
single_model_filenames = mminte.create_species_models(genome_ids, single_model_folder)

For reference, save the list of paths to single species models to a file.

In [13]:
with open(join(analysis_folder, 'single_model_filenames.txt'), 'w') as handle:
    handle.write('\n'.join(single_model_filenames)+'\n')

### Widget 4 - Create two species community models

In Widget 4, you create two species community models from single species models using the `create_interaction_models()` function. The first input parameter is a list of tuples with two elements, the paths to the single species model files in each pair. You can use all possible pairs from the list of single species models created in Widget 3 as shown in the cell below or select specific pairs by building the input list yourself.

In [14]:
pairs = mminte.get_all_pairs(single_model_filenames)
pairs[0]

('/Users/m097749/mminte_tutorial/single_models/1367212.3.json',
 '/Users/m097749/mminte_tutorial/single_models/1235835.3.json')

The second input parameter is the path to a folder for storing the two species community model files (created in "Getting Started").

The output is a list of paths to the two species community model files.

In [15]:
pair_model_filenames = mminte.create_interaction_models(pairs, pair_model_folder)

For reference, save the list of paths to the two species community models to a file.

In [16]:
with open(join(analysis_folder, 'pair_model_filenames.txt'), 'w') as handle:
    handle.write('\n'.join(pair_model_filenames)+'\n')

### Widget 5 - Calculate growth rates and evaluate interactions

In Widget 5, you calculate growth rates and evaluate interactions for the pairs in the microbial community using the `calculate_growth_rates()` function. To determine how the presence of one species in the community affects the growth of the other species, the two species community model is optimized for growth with the two species together and with each species alone under specific nutritient conditions. The type of interaction is set based on the outcomes identified in Table 1 of [Anoxic Conditions Promote Species-Specific Mutualism between Gut Microbes In Silico](http://aem.asm.org/content/81/12/4049.full). What happens between two species may be broadly divided into positive, negative, or no interaction. 

The first input parameter is the list of paths to two species community model files that were created in Widget 4. 

The second input parameter is a dictionary that defines the medium (or nutritient conditions) that the community is growing in. The medium dictionary is keyed by exchange reaction ID with the uptake rate as the value. A complete medium for the ModelSEED models used in this tutorial is in the test data included with the mminte package. 

In [17]:
complete_diet = mminte.read_diet_file(pkg_resources.resource_filename('mminte', 'test/data/ms_complete100.txt'))

The output is a data frame with details on the growth rates of the species in each pair and the type of interaction.

In [18]:
growth_rates = mminte.calculate_growth_rates(pair_model_filenames, complete_diet)

Each row in the `growth_rates` data frame, details the interaction between a pair in the microbial community and identifies the type of interaction. 

In [19]:
growth_rates[0:10]

,A_ID,B_ID,TYPE,TOGETHER,A_TOGETHER,B_TOGETHER,A_ALONE,B_ALONE,A_CHANGE,B_CHANGE
0,1367212.3,1235835.3,Parasitism,4.778327,4.778327,0.000000,2.389163,2.145181,1.000000,-1.000000
1,1367212.3,1165092.3,Parasitism,23.201545,13.772707,9.428838,2.389163,22.060714,4.764657,-0.572596
2,1367212.3,1313290.3,Parasitism,6.141120,0.000000,6.141120,2.389163,4.912896,-1.000000,0.250000
3,1367212.3,1469948.3,Parasitism,8.534269,6.613560,1.920709,2.389163,6.230712,1.768149,-0.691735
4,1367212.3,1000569.4,Parasitism,4.380133,4.380133,0.000000,2.389163,3.130046,0.833333,-1.000000
5,1367212.3,1414720.3,Parasitism,5.972908,5.972908,0.000000,2.389163,5.916418,1.500000,-1.000000
6,1367212.3,657315.3,Parasitism,21.732161,13.319827,8.412334,2.389163,19.120254,4.575101,-0.560030
7,1367212.3,525362.3,Parasitism,3.583745,3.583745,0.000000,2.389163,3.579748,0.500000,-1.000000
8,1367212.3,1156417.3,Amensalism,11.129313,2.145923,8.983390,2.389163,9.510915,-0.101810,-0.055465
9,1367212.3,180332.3,Amensalism,27.933158,0.000000,27.933158,2.389163,26.537878,-1.000000,0.052577


For reference, save the growth rates data frame to a file.

In [20]:
mminte.write_growth_rates_file(growth_rates, join(analysis_folder, 'growth_rates.csv'))

### Widget 6 - Visualize interactions in the community

In Widget 6, you visualize the interactions between pairs in the microbial community by creating a graph that represents the interaction network using `make_graph()`. 

The first input is the growth rates data frame created in Widget 5. The second input parameter is the similarity data frame created in Widget 2. The third input parameter is the correlations between OTUs used as input in Widget 1.

The output is a networkx Graph object where the nodes represent the different OTUs and the edges represent the interaction between OTUs in a pair. The shading of a node indicates how close the sequence of the OTU is to the sequence of the genome. The darker the node, the higher the similarity. The color of an edge indicates the kind of interaction predicted between the OTUs in a pair. A red edge indicates a negative interaction, a green edge indcates a positive interaction, and a gray edge indicates no interaction. The thickness of an edge indicates the strength of the correlation between the OTUs in a pair. The thicker the edge, the higher the correlation between the OTUs.

In [21]:
graph = mminte.make_graph(growth_rates, similarity, correlations)

You plot the graph using `plot_graph()` which opens a new window with the visualization. The default is a circular layout. If you want a different layout, you can use any of the plotting functions available in the networkx package.

In [22]:
%matplotlib
mminte.plot_graph(graph)

Using matplotlib backend: MacOSX


/Users/m097749/Envs/mminte-py27/lib/python2.7/site-packages/networkx/drawing/nx_pylab.py:126 MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
/Users/m097749/Envs/mminte-py27/lib/python2.7/site-packages/networkx/drawing/nx_pylab.py:138 MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
/Users/m097749/Envs/mminte-py27/lib/python2.7/site-packages/matplotlib/__init__.py:917 UserWarning: axes.hold is deprecated. Please remove it from your matplotlibrc and/or style files.
/Users/m097749/Envs/mminte-py27/lib/python2.7/site-packages/matplotlib/rcsetup.py:152 UserWarning: axes.hold is deprecated, will be removed in 3.0
